In [92]:
import pandas as pd

In [93]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [94]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [95]:
len(data)

174643

In [96]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [97]:
# hard-coded awards
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [98]:
import re

In [147]:
def reduce(line):
    # remove words "performance", "picture", "in", "a", "an", "any", "made", "for", "by", and all punctuations
    pattern = r'\bperformance\b|\bpicture\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
for award in awards:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['best', 'motion', 'drama']
['best', 'motion', 'musical', 'or', 'comedy']
['best', 'actress', 'motion', 'drama']
['best', 'actor', 'motion', 'drama']
['best', 'actress', 'motion', 'musical', 'or', 'comedy']
['best', 'actor', 'motion', 'musical', 'or', 'comedy']
['best', 'actress', 'supporting', 'role', 'motion']
['best', 'actor', 'supporting', 'role', 'motion']
['best', 'director', 'motion']
['best', 'screenplay', 'motion']
['best', 'motion', 'animated']
['best', 'motion', 'foreign', 'language']
['best', 'original', 'score', 'motion']
['best', 'original', 'song', 'motion']
['best', 'television', 'series', 'drama']
['best', 'television', 'series', 'musical', 'or', 'comedy']
['best', 'television', 'limited', 'series', 'or', 'motion']
['best', 'actress', 'limited', 'series', 'or', 'motion', 'television']
['best', 'actor', 'limited', 'series', 'or', 'motion', 'television']
['best', 'actress', 'television', 'series', 'drama']
['best', 'actor', 'television', 'series', 'drama']
['best', 'actr

In [183]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            num_words = len(award) - 1 - award.index('or') # number of words from "or" to the end of award name
            # test
            # print(len(award)-1, award.index('or'), num_words)
            award.remove('or')
            award_num_keywords_map[i] = len(award) - 1 - num_words # do not count "best" and words from "or" to the end of award name
        else:
            award_num_keywords_map[i] = len(award) - 1 # do not count "best"
        
#         if 'best' in award:
#             award.remove('best')

        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced


In [184]:
award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

[(0, 2),
 (1, 2),
 (2, 3),
 (3, 3),
 (4, 3),
 (5, 3),
 (6, 4),
 (7, 4),
 (8, 2),
 (9, 2),
 (10, 2),
 (11, 3),
 (12, 3),
 (13, 3),
 (14, 3),
 (15, 3),
 (16, 3),
 (17, 3),
 (18, 3),
 (19, 4),
 (20, 4),
 (21, 4),
 (22, 4),
 (23, 5),
 (24, 5),
 (25, 3)]

In [185]:
awards_reduced

[['best', 'motion', 'drama'],
 ['best', 'motion', 'musical', 'comedy'],
 ['best', 'actress', 'motion', 'drama'],
 ['best', 'actor', 'motion', 'drama'],
 ['best', 'actress', 'motion', 'musical', 'comedy'],
 ['best', 'actor', 'motion', 'musical', 'comedy'],
 ['best', 'actress', 'supporting', 'role', 'motion'],
 ['best', 'actor', 'supporting', 'role', 'motion'],
 ['best', 'director', 'motion'],
 ['best', 'screenplay', 'motion'],
 ['best', 'motion', 'animated'],
 ['best', 'motion', 'foreign', 'language'],
 ['best', 'original', 'score', 'motion'],
 ['best', 'original', 'song', 'motion'],
 ['best', 'television', 'series', 'drama'],
 ['best', 'television', 'series', 'musical', 'comedy'],
 ['best', 'television', 'limited', 'series', 'motion'],
 ['best', 'actress', 'limited', 'series', 'motion', 'television'],
 ['best', 'actor', 'limited', 'series', 'motion', 'television'],
 ['best', 'actress', 'television', 'series', 'drama'],
 ['best', 'actor', 'television', 'series', 'drama'],
 ['best', 'act

In [186]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
        
    return re.sub(pattern, ' ', line)

In [151]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [152]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [153]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

# test
cleanse("Hugh Jackman is so awesome!!! #goldenglobes")

'Hugh Jackman is so awesome    #goldenglobes'

In [221]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    matches = re.findall(pattern, line)
    for match in matches:
        line = re.sub(match, ' ', line)
    return line

In [243]:
%%time
num = 0
count = 0
pattern = re.compile(r'amy adams')
for tweet in data:
    # remove_retweet_prefix 
    line = remove_retweet_prefix(tweet)
    # remove hashtag
    line = remove_hashtag(line)
    # remove @...
    line = remove_at(line)
    # remove url
    line = remove_url(line)
    # remove punctuations
    line = cleanse(line)
    
    match = re.findall(pattern, line.lower())                  
    if match:
        tags = identify_entities(line)
        count += 1
        # check the first 10 cases
        if count < 10:
            print(num)
            print(match)
            print(tweet)
            print(tags.keys())
            print()
    num += 1
    
print(count)

295
['amy adams']
Orgulho da minha Amy Adams. &lt;3 #GoldenGlobes
dict_keys(['Amy Adams', '3'])

1240
['amy adams']
RT @jcpenney: Amy Adams looking beautiful in a blush @MarchesaFashion gown. #goldenglobes
dict_keys(['Amy Adams', ''])

1619
['amy adams']
#goldenglobes Amy Adams pale pink Marchesa mermaid dress is washing her out
dict_keys(['Amy Adams', 'Marchesa'])

1947
['amy adams']
RT @xoBetseyJohnson: #GoldenGlobes Amy Adams dress by Marchesa was heaven! Xox💋💋 http://t.co/Z0RsxThC
dict_keys(['Amy Adams', 'Marchesa', 'Xox'])

1957
['amy adams']
Da Amy Adams tbm. E por enquanto só delas... #GoldenGlobes #TNT
dict_keys(['Da Amy Adams', ''])

2298
['amy adams']
RT @arlene_bm: Amy Adams in @MarchesaFashion #GoldenGlobes 2013 http://t.co/akElt3IL”❤❤❤
dict_keys(['Amy Adams', '2013', ''])

2505
['amy adams']
RT @harpersbazaarES: Y si os preguntabáis quien firma el diseño sirena de Amy Adams, es de Marchesa ¿Qué os parece? #goldenglobes #redcarpet http://t.co/Pq5tcXCv
dict_keys(['de Amy Ada

In [154]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [155]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
    return tags

In [252]:
def find_award_winner(awards_reduced, award_index, verbose=False):
    print('Predicting for:', awards[award_index])    
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print('num keywords to match:', num_keywords_to_match)
    
    # add word bounary '\b' to prevent grabbing examples like "showing" and "wonder"
    pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)|(\b{0}\b)'.format(award[1]), re.IGNORECASE)
    entity_dict = {}
    
    num = 0
    for tweet in data:
        # remove_retweet_prefix 
        line = remove_retweet_prefix(tweet)
        # remove hashtag
        line = remove_hashtag(line)
        # remove @...
        line = remove_at(line)
        # remove url
        line = remove_url(line)
        # remove punctuations
        line = cleanse(line)
        
        
        # test for 'best actress in a motion picture comedy or musical'
#         if len(set(award).intersection(set(line.split()))) > 5:
        
        if len(set(award).intersection(set(line.split()))) > num_keywords_to_match:
            
            match = re.findall(pattern, line.lower())
            if match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(tweet)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    if len(entity) > 1:
                        if entity not in entity_dict:
                            entity_dict[entity] = 1
                        else:
                            entity_dict[entity] += 1
                num += 1
            
    print('num of matches:', num)
    return entity_dict

In [253]:
%%time
# test
entity_dict = find_award_winner(awards_reduced, 0, verbose=True)

Predicting for: Best Motion Picture - Drama
award name reduced: ['best', 'motion', 'drama']
num keywords to match: 2
[('win', '', '', ''), ('', '', 'best', ''), ('', '', '', 'motion')]
Jessica Chastain wins #Goldenglobes for best actress - drama, motion pitcure for Zero Dark Thirty
dict_keys(['Jessica Chastain', 'Zero'])

[('', '', 'best', ''), ('', '', '', 'motion')]
RT @TODAY_ent: The award for best actress in a motion picture drama goes to Jessica Chastain for @ZeroDarkThirty http://t.co/BshI8XDL #GoldenGlobes
dict_keys(['Jessica Chastain', ''])

[('win', '', '', ''), ('', '', 'best', ''), ('', '', '', 'motion')]
Jessica Chastain wins best actress in a motion picture, drama, for 'Zero Dark Thirty' at the Golden Globes - live broadcast
dict_keys(['Jessica Chastain', "for '", 'Zero', "Dark Thirty '", 'the Golden Globes'])

[('', '', 'best', ''), ('', '', '', 'motion')]
RT @TODAY_ent: The award for best actress in a motion picture drama goes to Jessica Chastain for @ZeroDarkThirty http

# Predictions for Award Winner

In [254]:
%%time
for key in award_num_keywords_map.keys():
    print(sorted(find_award_winner(awards_reduced, key, verbose=False).items(), key=lambda pair: pair[1], reverse=True)[:100])
    print()

Predicting for: Best Motion Picture - Drama
award name reduced: ['best', 'motion', 'drama']
num keywords to match: 2
num of matches: 50
[('Argo', 12), ('Latest', 10), ('Lincoln', 9), ('Jessica Chastain', 7), ('Daniel Day Lewis', 6), ('Julia Roberts', 6), ('Ben Affleck', 4), ('Zero', 3), ("for '", 2), ("Dark Thirty '", 2), ('the Golden Globes', 2), ('Julia', 2), ('Lancome Ambassadress', 2), ('Spielberg', 1), ('Affleck', 1), ('one', 1), ('Roberts', 1), ("the Golden Globes         ''", 1)]

Predicting for: Best Motion Picture - Musical or Comedy
award name reduced: ['best', 'motion', 'musical', 'comedy']
num keywords to match: 2
num of matches: 394
[('Les Miserables', 127), ('Hugh Jackman', 113), ('Jennifer Lawrence', 39), ("Les Miserables '", 24), ('the Golden Globes   more', 22), ('Don Cheadle', 19), ('Golden Globe', 19), ('Eagles', 18), ('Bradley Cooper', 18), ('VW', 18), ('Lena Dunham', 14), ('WINNER', 10), ('Girls', 9), ('Les Mis', 6), ('night', 6), ('Bill Murray', 6), ('Miserables',

num of matches: 19
[('Don Cheadle', 14)]

Predicting for: Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television
award name reduced: ['best', 'actress', 'supporting', 'role', 'series', 'limited', 'motion', 'television']
num keywords to match: 5
num of matches: 0
[]

Predicting for: Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television
award name reduced: ['best', 'actor', 'supporting', 'role', 'series', 'limited', 'motion', 'television']
num keywords to match: 5
num of matches: 0
[]

Predicting for: Cecil B. deMille Award
award name reduced: ['cecil', 'b', 'demille', 'award']
num keywords to match: 3
num of matches: 0
[]

CPU times: user 3min 58s, sys: 16.7 s, total: 4min 14s
Wall time: 2min 18s
